 ##앙상블 종류

 ###1. 투표방식
- bagging : 랜덤포레스트
- voting : 소프트보팅, 하드보팅

In [1]:
# votingclassifier 이용
# 매개변수 : estimators, voting 방식

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline

In [3]:
wine = pd.read_csv('data/wine.csv')

y = wine['color']
X = wine.drop('color',axis = 1)

In [4]:
ohe = OneHotEncoder()
quality = ohe.fit_transform(np.array(X['quality']).reshape(-1,1)).toarray()
quality

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [5]:
ohe.get_feature_names()

array(['x0_A', 'x0_B', 'x0_C'], dtype=object)

In [6]:
X = pd.concat([X, pd.DataFrame(quality, columns = ohe.get_feature_names())], axis = 1)
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,x0_A,x0_B,x0_C
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,C,0.0,0.0,1.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,C,0.0,0.0,1.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,C,0.0,0.0,1.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,B,0.0,1.0,0.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,C,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,B,0.0,1.0,0.0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,C,0.0,0.0,1.0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,B,0.0,1.0,0.0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,A,1.0,0.0,0.0


In [7]:
X.drop('quality', axis = 1, inplace = True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, random_state = 1)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier #from 절에는 모듈
from sklearn.pipeline import make_pipeline

In [10]:
knn = KNeighborsClassifier(n_neighbors = 5)
rf = RandomForestClassifier(n_estimators = 200)
svc = SVC(C = 0.1, gamma = 0.1, probability = True)

knn_pipe = make_pipeline(StandardScaler(), knn)
svc_pipe = make_pipeline(StandardScaler(), svc)
estimators = [('knn', knn_pipe),
              
             ('random_forest', rf),
             ('svm', svc_pipe)]

voting = VotingClassifier(estimators) # (이름, 객채명)

In [11]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def print_metrics(y,y_pred, title = None):
    if title:
        print(title)
        metrics = f'정확도:{accuracy_score(y, y_pred)}, 재현율:{recall_score(y,y_pred)}, 정밀도:{precision_score(y,y_pred)}, f1_score :{f1_score(y, y_pred)}'
        print(metrics)

In [12]:
voting.fit(X_train, y_train)

VotingClassifier(estimators=[('knn',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier())])),
                             ('random_forest',
                              RandomForestClassifier(n_estimators=200)),
                             ('svm',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('svc',
                                               SVC(C=0.1, gamma=0.1,
                                                   probability=True))]))])

In [13]:
pred_train = voting.predict(X_train)
pred_test = voting.predict(X_test)

In [14]:
print_metrics(y_train, pred_train, 'train')

train
정확도:0.9956896551724138, 재현율:0.9858215179316097, 정밀도:0.9966273187183811, f1_score :0.991194968553459


In [15]:
print_metrics(y_test, pred_test, 'test')

test
정확도:0.9969230769230769, 재현율:0.9925, 정밀도:0.9949874686716792, f1_score :0.9937421777221527


In [16]:
from sklearn.ensemble import VotingClassifier

In [17]:
voting_soft = VotingClassifier(estimators,
                              voting= 'soft')

In [18]:
voting_soft.fit(X_train,y_train)

VotingClassifier(estimators=[('knn',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier())])),
                             ('random_forest',
                              RandomForestClassifier(n_estimators=200)),
                             ('svm',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('svc',
                                               SVC(C=0.1, gamma=0.1,
                                                   probability=True))]))],
                 voting='soft')

In [19]:
pred_train_soft = voting_soft.predict(X_train)
pred_test_soft = voting_soft.predict(X_test)

In [20]:
print_metrics(y_test, pred_test_soft, 'test, soft보팅')

test, soft보팅
정확도:0.9963076923076923, 재현율:0.9925, 정밀도:0.9925, f1_score :0.9925


In [1]:
#랜덤 포레스트 연습
from sklearn.datasets import load_breast_cancer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
X,y = load_breast_cancer(return_X_y = True)
X.shape, y.shape

((569, 30), (569,))

In [4]:
X_train, X_test, y_train,y_test = train_test_split(X,y, stratify = y, random_state = 1)
X_train.shape, y_train.shape

((426, 30), (426,))

In [6]:
rf = RandomForestClassifier(n_estimators = 200,
                           max_features = 15,
                           max_depth = 5,
                           n_jobs = -1,
                           random_state = 1)

In [7]:
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, max_features=15, n_estimators=200,
                       n_jobs=-1, random_state=1)

In [10]:
pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

In [11]:
accuracy_score(y_test, pred_test)

0.951048951048951

In [12]:
d = load_breast_cancer()

In [13]:
ri = rf.feature_importances_

In [15]:
print(ri)

[0.00463955 0.01384115 0.00379319 0.01078856 0.00189817 0.00304166
 0.02963914 0.11022216 0.0018731  0.00176271 0.00509571 0.0031751
 0.0031342  0.01818324 0.0031701  0.00189934 0.00247902 0.00220542
 0.00237803 0.00454188 0.11275583 0.02374541 0.20790111 0.13835278
 0.00696561 0.00800956 0.01499553 0.25203332 0.00509586 0.00238357]


In [17]:
fi_s = pd.Series(ri, index = d['feature_names'])
fi_s

mean radius                0.004640
mean texture               0.013841
mean perimeter             0.003793
mean area                  0.010789
mean smoothness            0.001898
mean compactness           0.003042
mean concavity             0.029639
mean concave points        0.110222
mean symmetry              0.001873
mean fractal dimension     0.001763
radius error               0.005096
texture error              0.003175
perimeter error            0.003134
area error                 0.018183
smoothness error           0.003170
compactness error          0.001899
concavity error            0.002479
concave points error       0.002205
symmetry error             0.002378
fractal dimension error    0.004542
worst radius               0.112756
worst texture              0.023745
worst perimeter            0.207901
worst area                 0.138353
worst smoothness           0.006966
worst compactness          0.008010
worst concavity            0.014996
worst concave points       0

# 앙상블 부스팅 

In [18]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [19]:
data = load_breast_cancer()
X,y =data['data'], data['target']

In [20]:
X_train,X_test, y_train, y_test = train_test_split(X,y , stratify = y, random_state = 1)
y_train.shape, y_test.shape

((426,), (143,))

In [21]:
gb = GradientBoostingClassifier(random_state = 1)
gb.fit(X_train,y_train)

GradientBoostingClassifier(random_state=1)

In [22]:
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

In [23]:
#검증
accuracy_score(y_train, pred_train), accuracy_score(y_test,pred_test)
#데이터 양이 워낙 적어 과적합 날 수 밖에
# 학습데이터의 정확도 떨어지면 어쨌든 모델이 단순화되는, 일반화되는 것에 가깝

(1.0, 0.958041958041958)

In [24]:
import pandas as pd
fi = gb.feature_importances_

In [25]:
fi_s = pd.Series(fi, index = data['feature_names'])

In [26]:
fi_s.sort_values(ascending = False)

worst radius               0.383760
worst concave points       0.285992
worst perimeter            0.130660
mean concave points        0.046544
worst area                 0.042472
worst texture              0.041142
worst concavity            0.013033
area error                 0.010910
mean texture               0.009101
mean concavity             0.007719
radius error               0.004902
concavity error            0.003397
worst fractal dimension    0.002838
mean area                  0.002552
worst symmetry             0.002546
fractal dimension error    0.002532
mean compactness           0.001566
compactness error          0.001386
mean perimeter             0.001368
smoothness error           0.001340
symmetry error             0.001265
perimeter error            0.000813
mean radius                0.000701
mean fractal dimension     0.000625
texture error              0.000498
worst smoothness           0.000144
worst compactness          0.000097
mean smoothness            0

In [27]:
#그리드 서치 이용

In [28]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [30]:
param={
    'n_estimators':[100,200,300,400,500],
    'learning_rate':[0.001,0.05,0.01,0.5, 0.1],
    'max_depth':range(1,5),
    'subsample':[0.5,0.7,1]
}

gb = GradientBoostingClassifier(random_state = 1)

gs = GridSearchCV(gb, 
                 param_grid = param,
                 cv = 3,
                 scoring = 'accuracy',
                 n_jobs = -1)

In [31]:
gs.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=1),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.05, 0.01, 0.5, 0.1],
                         'max_depth': range(1, 5),
                         'n_estimators': [100, 200, 300, 400, 500],
                         'subsample': [0.5, 0.7, 1]},
             scoring='accuracy')

In [32]:
gs.best_params_

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.5}

In [33]:
result_df = pd.DataFrame(gs.cv_results_)
result_df.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
195,0.276503,0.009870,0.001522,0.000630,0.5,2,100,0.5,"{'learning_rate': 0.5, 'max_depth': 2, 'n_esti...",0.964789,0.978873,0.978873,0.974178,0.006640,1
207,1.428189,0.027914,0.003414,0.000457,0.5,2,500,0.5,"{'learning_rate': 0.5, 'max_depth': 2, 'n_esti...",0.964789,0.978873,0.978873,0.974178,0.006640,1
109,1.188301,0.015864,0.003251,0.000892,0.05,4,200,0.7,"{'learning_rate': 0.05, 'max_depth': 4, 'n_est...",0.971831,0.964789,0.985915,0.974178,0.008783,1
289,1.063881,0.020917,0.002716,0.000512,0.1,4,200,0.7,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.964789,0.971831,0.985915,0.974178,0.008783,1
263,1.207778,0.019371,0.005600,0.001486,0.1,2,300,1,"{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",0.957746,0.971831,0.992958,0.974178,0.014470,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,1.801691,0.026239,0.006734,0.006118,0.001,3,200,0.7,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",0.626761,0.626761,0.626761,0.626761,0.000000,277
30,0.700589,0.063664,0.001811,0.000612,0.001,3,100,0.5,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",0.626761,0.626761,0.626761,0.626761,0.000000,277
31,0.864712,0.117341,0.002831,0.000947,0.001,3,100,0.7,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",0.626761,0.626761,0.626761,0.626761,0.000000,277
33,1.068015,0.015749,0.020691,0.022959,0.001,3,200,0.5,"{'learning_rate': 0.001, 'max_depth': 3, 'n_es...",0.626761,0.626761,0.626761,0.626761,0.000000,277


In [34]:
pred_test = gs.predict(X_test)
accuracy_score(y_test, pred_test)

0.951048951048951

In [35]:
model = gs.best_estimator_
model.feature_importances_

array([0.00278067, 0.02227644, 0.00375299, 0.00889619, 0.00402174,
       0.00660803, 0.01243995, 0.0678632 , 0.00579283, 0.00286154,
       0.00726701, 0.00815831, 0.00620805, 0.02418745, 0.0072363 ,
       0.0008025 , 0.00222562, 0.00133842, 0.00125631, 0.00766437,
       0.16701911, 0.02320413, 0.21548976, 0.09680912, 0.00123604,
       0.01636319, 0.01633569, 0.23862063, 0.01413482, 0.0071496 ])

In [ ]:
#XGBOOST
#Gradientboost의 느린속도, 과적합을 제어할 수 있는 규제 제공해 성능 높인 버전
# 분류, 회귀 다 지원
#매개변수 : learning_rate, n_estimators, max_depth

In [37]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators = 200,
                   learning_rate = 0.5,
                   max_depth = 2)

In [38]:
xgb.fit(X_train, y_train)

[17:30:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Playdata\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)